# Step 2: Pre-processing - Calculating yearly Mean 📅

The goal is to calculate the **yearly mean** of the variables from the **Baltic Sea Wave Hindcast** dataset, which is available only in **hourly resolution**. Each day has a separate file, and each file is around **200MB** in size. If I wanted to download the entire dataset and calculate the mean, I would need **657GB** of storage! Here's the math: (*365 * 200 * (2024-2015) = 657,000 MB = 657 GB*)... 

Since this isn't feasible, I will use a technique often employed in survey sampling. Instead of downloading the entire dataset, I will download a **random sample** of the data and calculate the mean based on this sample. To make it more manageable, I’ll assume I download **10%** of the dataset, which means downloading **328 files** (*365 * 9 * 0.10 = 328*).

#### 📚 Required Libraries
To carry out this process, you'll need the following libraries:
- **`pandas`**: For reading, manipulating, and analyzing the data. 
- **`random`**: For generating random samples from the data. 
- **`requests`**: For interacting with the API and downloading files. 
- **`netCDF4`**: For working with NetCDF files, the format used by this dataset.

### 🛠️ Steps:
1. **Generate a Random List of Dates**: Create a list of random dates to represent 10% of the total dataset.
2. **Download the Files**: Using the list of random dates, download the corresponding files from the API. 
3. **Apply the Spatial Mask**: Use a spatial filter to restrict the data to the region of interest. Fisrt, I used the `livinglab_geometry` as a mask but it creates border issues so I filter again using the bounding box of the `livinglab_geometry`. Spatially filtering is useless because of resampling.
4. **Calculate the Annual Mean**: By combining the datasets, calculate the yearly mean.
5. **Recalculate the Annual Mean with the Boundingbox Mask** : ...

This method significantly reduces the amount of data needed while still providing a statistically valid approximation.

______


### 🛠️ Creating a Random List of Dates 

Let’s start by generating a random list of **328 dates**. This list will determine which files to download and will be the foundation for calculating the mean based on a representative sample of the data.

In [5]:
import pandas as pd
import random

In [4]:
# Generate the full list of dates from January 1, 2015 to December 31, 2023
start_date = "2015-01-01"
end_date = "2023-12-31"

# Generate the date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Randomly sample 328 dates from this range
sampled_dates = random.sample(list(date_range), 328)

sampled_dates.sort()

We can now save our random dates in a csv file. Once I saved it, I make sure not to overwrite the file.

In [14]:
# pd.Series(sampled_dates).to_csv('sampled_dates.csv', index=False)

### 🛠️ Dowloading the files

I don't want to dowload this 328 files one by one. I can use the API of Copernicus Marine Data Store. It is possible to create a list of files or dowload the files one by one. You can find more informations on the [Copernicus Marine API page](https://help.marine.copernicus.eu/en/articles/8286883-copernicus-marine-toolbox-api-get-original-files#h_4cb9923744).

⚠️ The dowloaded files are not saved in the GitHub repositary because it was too heavy (56GB). However, the spatial filtered annual mean version is saved.

In [7]:
import copernicusmarine as cm

To dowload a dataset of the Copernicus Marine API, you have to use its ID. You can found its ID in the Data access page, in grey on the Dataset column. You can click on "📑​" to copy it. It will display a message : "Copied dataset ID to your clipboard".

In [6]:
datasetID = 'cmems_mod_bal_wav_my_PT1H-i'
sampled_dates = pd.read_csv('sampled_dates.csv', parse_dates=[0]).iloc[:, 0].tolist()

You need to create an account to dowload the dataset. Below is the method to dowload a specific file. (I have dowloaded one by one).

In [ ]:
i=0
for date in sampled_dates:
    i += 1
    year = date.year
    month = date.month 
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    cm.get(
        username = 'your_username',
        password = 'your_password',
        dataset_id  = datasetID,
        filter=f'*{year}{month}{day}*'
        )
    print(f'Dowloading {i} / 328 files for {year}-{month}-{day}')

INFO - 2025-07-29T12:56:45Z - Selected dataset version: "202411"
INFO - 2025-07-29T12:56:45Z - Selected dataset part: "default"
INFO - 2025-07-29T12:56:46Z - Listing files on remote server...
17it [00:09,  1.84it/s]


Dowloading 249 / 328 files for 2022-01-07


INFO - 2025-07-29T12:57:15Z - Selected dataset version: "202411"
INFO - 2025-07-29T12:57:15Z - Selected dataset part: "default"
INFO - 2025-07-29T12:57:16Z - Listing files on remote server...
17it [00:09,  1.70it/s]


Dowloading 250 / 328 files for 2022-01-10


INFO - 2025-07-29T12:57:49Z - Selected dataset version: "202411"
INFO - 2025-07-29T12:57:49Z - Selected dataset part: "default"
INFO - 2025-07-29T12:57:49Z - Listing files on remote server...
17it [00:09,  1.80it/s]


...

Dowloading 326 / 328 files for 2023-12-05


INFO - 2025-07-29T13:43:01Z - Selected dataset version: "202411"
INFO - 2025-07-29T13:43:01Z - Selected dataset part: "default"
INFO - 2025-07-29T13:43:02Z - Listing files on remote server...
17it [00:09,  1.71it/s]


Dowloading 327 / 328 files for 2023-12-19


INFO - 2025-07-29T13:43:41Z - Selected dataset version: "202411"
INFO - 2025-07-29T13:43:41Z - Selected dataset part: "default"
INFO - 2025-07-29T13:43:42Z - Listing files on remote server...
17it [00:10,  1.61it/s]

Dowloading 328 / 328 files for 2023-12-28


### 🛠️ Spatial filtering with the `livinglab_geometry` - USELESS

Before kwnowing this step would be useless, I filtered the files using the same method I used in the 02_Spatial_filtering notebook. Big mistake : I removed the dowloaded files to reduce the space it takes on my computer.

In [1]:
import xarray as xr
import numpy as np
from shapely.geometry import shape
import geopandas as gpd
import os
import time
import gc

In [2]:
livinglab_geometry = gpd.read_file('livinglab_west.json')
path = 'BALTICSEA_MULTIYEAR_WAV_003_015\cmems_mod_bal_wav_my_PT1H-i_202411'
input_file = f'{path}/2015/01/CMEMS_BAL_WAV_MY_2015010201.nc'
ds = xr.open_dataset(input_file)
ds

<xarray.Dataset> Size: 1GB
Dimensions:    (lat: 775, lon: 764, time: 24)
Coordinates:
  * lat        (lat) float32 3kB 53.01 53.02 53.04 53.06 ... 65.87 65.89 65.91
  * lon        (lon) float32 3kB 9.014 9.042 9.069 9.097 ... 30.15 30.18 30.21
  * time       (time) datetime64[ns] 192B 2015-01-02T01:00:00 ... 2015-01-03
Data variables: (12/19)
    VHM0       (time, lat, lon) float32 57MB ...
    VTPK       (time, lat, lon) float32 57MB ...
    VTM10      (time, lat, lon) float32 57MB ...
    VTM02      (time, lat, lon) float32 57MB ...
    VMDR       (time, lat, lon) float32 57MB ...
    VHM0_WW    (time, lat, lon) float32 57MB ...
    ...         ...
    VTM01_SW2  (time, lat, lon) float32 57MB ...
    VMDR_SW2   (time, lat, lon) float32 57MB ...
    VSDX       (time, lat, lon) float32 57MB ...
    VSDY       (time, lat, lon) float32 57MB ...
    VMXL       (time, lat, lon) float32 57MB ...
    VCMX       (time, lat, lon) float32 57MB ...
Attributes:
    Conventions:            CF-1.0
    source:                 FMI-WAM_CMEMS
    institution:            Baltic MFC, PU Finnish Meteorological Institute
    title:                  2D - Hourly Instantaneous
    cmems_product_id:       BALTICSEA_MULTIYEAR_WAV_003_015
    westernmost_longitude:  9.0138
    easternmost_longitude:  30.2080
    southernmost_latitude:  53.0083
    northernmost_latitude:  65.9081
    grid_resolution:        1 nautical mile (ie. 0.0167 degrees northward; 0....

In [ ]:
i = 0
for date in sampled_dates:
    i += 1
    # If you want to skip some dates
    # if date < pd.Timestamp('2017-01-29'): 
    #     continue
    year = date.year
    month = date.month
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    input_file_name = f'CMEMS_BAL_WAV_MY_{year}{month}{day}01.nc'
    output_file_name = f'filtered_{input_file_name}'
    input_file = f'{path}/{year}/{month}/{input_file_name}'
    output_file = f'{path}/{year}/{month}/{output_file_name}'

    ds = xr.open_dataset(input_file)
    lat = ds['lat'].values
    lon = ds['lon'].values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    coords = np.column_stack((lon_grid.ravel(), lat_grid.ravel()))
    mask = np.array([livinglab_geometry.contains(shape({'type': 'Point', 'coordinates': (lon, lat)})) for lon, lat in coords])
    mask = mask.reshape(lon_grid.shape)
    mask_da = xr.DataArray(mask, coords=[ds['lat'], ds['lon']], dims=["lat", "lon"])
    filtered_data = ds.where(mask_da, drop=True) 
    
    try:
        filtered_data.to_netcdf(output_file)
    finally:
        ds.close()
        filtered_data.close()
        del ds
        del filtered_data
        gc.collect()
        time.sleep(1)

    # Try to delete the file, retrying if necessary
    for attempt in range(5):
        try:
            if os.path.exists(output_file):
                os.remove(input_file)
            break
        except PermissionError:
            time.sleep(1)
    print(f'File {i} / 328 processed: {output_file_name}')

File 70 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017020801.nc
File 71 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017022001.nc
File 72 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017032801.nc
File 73 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017040101.nc
File 74 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017041401.nc
File 75 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017041701.nc
File 76 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017042301.nc
File 77 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017043001.nc
File 78 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017051001.nc
File 79 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017051101.nc
File 80 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017051301.nc
File 81 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017051401.nc
File 82 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017052601.nc
File 83 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017060601.nc
File 84 / 328 processed: filtered_CMEMS_BAL_WAV_MY_2017062601.nc
File 85 / 328 processed: 

### 🛠️ Calculating the annual mean of the filtered dataset (`livinglab_west` geometry) - USELESS

I will merge the datasets per year and caculate the mean.

In [ ]:
# Filter the sampled_dates list for the specified year
year_to_filter = 2015
filtered_dates = [date for date in sampled_dates if date.year == year_to_filter]
files = [f'{path}/{date.year}/{date.month:02d}/filtered_CMEMS_BAL_WAV_MY_{date.year:04d}{date.month:02d}{date.day:02d}01.nc' for date in filtered_dates]
# Open the datasets and concatenate them along the time dimension
datasets = [xr.open_dataset(file) for file in files]
# Concatenate the datasets along the 'time' dimension
combined_dataset = xr.concat(datasets, dim='time')
# Calculate the mean for each variable across time for the year
yearly_mean = combined_dataset.mean(dim='time')
yearly_mean

<xarray.Dataset> Size: 2MB
Dimensions:    (lat: 225, lon: 109)
Coordinates:
  * lat        (lat) float32 900B 55.34 55.36 55.37 55.39 ... 59.07 59.09 59.11
  * lon        (lon) float32 436B 10.04 10.07 10.1 10.12 ... 12.99 13.01 13.04
Data variables: (12/19)
    VHM0       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VTPK       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VTM10      (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VTM02      (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VMDR       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VHM0_WW    (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    ...         ...
    VTM01_SW2  (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VMDR_SW2   (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VSDX       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VSDY       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VMXL       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan
    VCMX       (lat, lon) float32 98kB nan nan nan nan nan ... nan nan nan nan

In [ ]:
for year_to_filter in range(2015,2024):
    filtered_dates = [date for date in sampled_dates if date.year == year_to_filter]
    files = [f'{path}/{date.year}/{date.month:02d}/filtered_CMEMS_BAL_WAV_MY_{date.year:04d}{date.month:02d}{date.day:02d}01.nc' for date in filtered_dates]
    # Open the datasets and concatenate them along the time dimension
    datasets = [xr.open_dataset(file) for file in files]
    # Concatenate the datasets along the 'time' dimension
    combined_dataset = xr.concat(datasets, dim='time')
    # Calculate the mean for each variable across time for the year
    yearly_mean = combined_dataset.mean(dim='time')
    # Save the yearly mean to a new NetCDF file
    yearly_mean.to_netcdf(f'{path}/{year_to_filter}/yearly_mean_CMEMS_BAL_WAV_MY_{year_to_filter}.nc')
    print(f'Yearly mean for {year_to_filter} saved.')

Yearly mean for 2015 saved.
Yearly mean for 2016 saved.
Yearly mean for 2017 saved.
Yearly mean for 2018 saved.
Yearly mean for 2019 saved.
Yearly mean for 2020 saved.
Yearly mean for 2021 saved.
Yearly mean for 2022 saved.
Yearly mean for 2023 saved.


### 🛠️ Calculating the annual mean of the filtered dataset (bounding box geometry)

I need to recalculate the yearly mean with a different mask. Using the Living Lab West mask create issues at the border of the geometry. Consequensly, there is a loss of data at the border... because I am working in shalow water, the border data are very important ! I will create and use the boundingbox of the living lab geometry as a mask. Let's start by the year 2028.

In [ ]:
i=0
for date in sampled_dates:
    i += 1
    year = date.year
    if year != 2018:
        continue
    month = date.month 
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    cm.get(
        username = 'your_username',
        password = 'your_password',
        dataset_id  = datasetID,
        filter=f'*{year}{month}{day}*'
        )
    print(f'Dowloading {i} / 328 files for {year}-{month}-{day}')

In [7]:
i = 0
for date in sampled_dates:
    i += 1
    # If you want to skip some dates
    # if date < pd.Timestamp('2017-01-29'): 
    #     continue
    year = date.year
    if year != 2018:
        continue
    month = date.month
    if month < 10:
        month = f'0{month}'             
    day = date.day
    if day < 10:
        day = f'0{day}'
    input_file_name = f'CMEMS_BAL_WAV_MY_{year}{month}{day}01.nc'
    output_file_name = f'bounding_box_filtered_{input_file_name}'
    input_file = f'{path}/{year}/{input_file_name}'
    output_file = f'{path}/{year}/{output_file_name}'

    ds = xr.open_dataset(input_file)
    lat = ds['lat'].values
    lon = ds['lon'].values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    coords = np.column_stack((lon_grid.ravel(), lat_grid.ravel()))
    # Get the bounding box of the livinglab_geometry
    minx, miny, maxx, maxy = livinglab_geometry.total_bounds
    # Create a mask using the bounding box
    mask = ((lat_grid >= miny) & (lat_grid <= maxy) & (lon_grid >= minx) & (lon_grid <= maxx))
    mask_da = xr.DataArray(mask, coords=[ds['lat'], ds['lon']], dims=["lat", "lon"])
    filtered_data = ds.where(mask_da, drop=True) 
    
    try:
        filtered_data.to_netcdf(output_file)
    finally:
        ds.close()
        filtered_data.close()
        del ds
        del filtered_data
        gc.collect()
        time.sleep(1)

    # Try to delete the file, retrying if necessary
    for attempt in range(5):
        try:
            if os.path.exists(output_file):
                os.remove(input_file)
            break
        except PermissionError:
            time.sleep(1)
    print(f'File {i} / 328 processed: {output_file_name}')

File 104 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018011201.nc
File 105 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018012901.nc
File 106 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018020101.nc
File 107 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018021501.nc
File 108 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018022201.nc
File 109 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018031201.nc
File 110 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018031901.nc
File 111 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018040401.nc
File 112 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018041301.nc
File 113 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018041801.nc
File 114 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018042301.nc
File 115 / 328 processed: bounding_box_filtered_CMEMS_BAL_WAV_MY_2018050801.nc
File 116 / 328 processed: bounding_box_filtered_CMEM

In [8]:
# Filter the sampled_dates list for the specified year
year_to_filter = 2018
filtered_dates = [date for date in sampled_dates if date.year == year_to_filter]
files = [f'{path}/{year_to_filter}/bounding_box_filtered_CMEMS_BAL_WAV_MY_{year_to_filter}{date.month:02d}{date.day:02d}01.nc' for date in filtered_dates]
# Open the datasets and concatenate them along the time dimension
datasets = [xr.open_dataset(file) for file in files]
# Concatenate the datasets along the 'time' dimension
combined_dataset = xr.concat(datasets, dim='time')
# Calculate the mean for each variable across time for the year
yearly_mean = combined_dataset.mean(dim='time')
yearly_mean

<xarray.Dataset> Size: 2MB
Dimensions:    (lat: 227, lon: 109)
Coordinates:
  * lat        (lat) float32 908B 55.34 55.36 55.37 55.39 ... 59.07 59.09 59.11
  * lon        (lon) float32 436B 10.04 10.07 10.1 10.12 ... 12.99 13.01 13.04
Data variables: (12/19)
    VHM0       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTPK       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTM10      (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VTM02      (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMDR       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VHM0_WW    (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    ...         ...
    VTM01_SW2  (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMDR_SW2   (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VSDX       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VSDY       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VMXL       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan
    VCMX       (lat, lon) float32 99kB nan nan nan nan nan ... nan nan nan nan